In [ ]:
#| default_exp cell_display


In [ ]:
#| export
from __future__ import annotations


# Cell display
> Helpers for output cells display


# Prologue

In [ ]:
#| export

from IPython.display import DisplayHandle
from IPython.display import HTML


In [ ]:
#| export

from bridget.helpers import id_gen
from bridget.helpers import Script


In [ ]:
import inspect
from inspect import Parameter
from pathlib import Path
from typing import Protocol

import anywidget
import fastcore.all as FC
from fastcore.test import *
from fastcore.xml import to_xml
from IPython.display import display
from IPython.display import Javascript
from IPython.display import Markdown
from olio.common import setup_console


----


In [ ]:
#| exporti

new_id = id_gen()


In [ ]:
console, cprint = setup_console(140)


In [ ]:
# needed for vfile:
%load_ext anywidget


In [ ]:
%env ANYWIDGET_HMR=0

env: ANYWIDGET_HMR=0


----

# autoid (not used)

In [ ]:
%%vfile autoid.js

// debugger;
function autodel(id) {
    // debugger;
    const me = document.querySelector(`#${id}`);
    const parent = me.parentElement;
    // parent.append(`test ${id}`);
    window.setTimeout(() => {{ 
        me.remove(); 
        parent.setAttribute("id", `output-${id}`);
        console.log('deleted script', id); 
    }}, 100);
    console.log("test script", id);
}


In [ ]:
sc = '''
// debugger;
function autodel(id) {
    // debugger;
    const me = document.querySelector(`#${id}`);
    const parent = me.parentElement;
    // parent.append(`test ${id}`);
    window.setTimeout(() => {{ 
        me.remove(); 
        parent.setAttribute("id", `output-${id}`);
        console.log('deleted script', id); 
    }}, 100);
    console.log("test script", id);
}
'''

# autodelscr = Script(sc)
autodelscr = Script('vfile:autoid.js')
HTML(to_xml(autodelscr))

In [ ]:
sc = '''
// debugger;
autodel("{0}");
'''

idx = new_id()
scr = Script(sc.format(idx), id=idx, type='module')
HTML(to_xml(scr)+'aaaa')

In [ ]:
sc = '''
// debugger;
autodel("{0}");
'''
def autoid(idx=None):
    idx = idx or new_id()
    return Script(sc.format(idx), id=idx, type='module'), idx


In [ ]:
HTML('aaaaa'+to_xml(autoid()[0]))


In [ ]:
# %%HTML
#   <script id="asdfg-12345">
#     debugger;
#     me().attribute('id', 'output-asdfg-12345');
#     setTimeout(el => { el.remove(); }, 100, me('#asdfg-12345'))
#     // autodel('asdfg-12345');
#   </script>
#   <div class="uploader">aaaa</div>


In [ ]:
#| export

__autoid_scr = '''
//debugger;
me().attribute('id', 'output-{0}').classAdd('bridge');
setTimeout(el => {{ el.remove(); }}, 100, me('#{0}'))
'''
def autoid(idx=None):
    idx = idx or new_id()
    return Script(__autoid_scr.format(idx), id=idx), idx


In [ ]:
scr,idx = autoid()
dhdl = DisplayHandle(idx)
dhdl.display(HTML('bbbb'+to_xml(scr)))


In [ ]:
dhdl.update(HTML('cccc'))

# DisplayId

An attempt to provide a IPython display wrapper that automatically handles the display ID to allow us to target especific cells. Not working, for now it's essentially just IPython display.

In [ ]:
#| export

class DisplayId(DisplayHandle):
    def __init__(self, display_id=None):
        super().__init__(display_id or new_id())
        self._contents = None
        self._sc = ''
        # self._sc = to_xml(autoid(self.display_id)[0]) if bridge_cfg.auto_id else ''

    def display(self, obj='', **kwargs):
        from IPython.display import display
        self._contents = str(obj)
        display(HTML(self._contents + self._sc), display_id=self.display_id, **kwargs)

    def update(self, obj='', **kwargs):
        kwargs['update'] = True
        self.display(obj, **kwargs)

    def contents(self):
        return self._contents


In [ ]:
dhdl = DisplayId()
dhdl.display('dddd')
test_eq(dhdl.contents(), 'dddd')

In [ ]:
dhdl.update('eeee')
test_eq(dhdl.contents(), 'eeee')


In [ ]:
dhdl.update()

In [ ]:
dhdl = DisplayId(display_id='faf-sadfsfa-8434r2jhknjlsd-45hj')
dhdl.display('gggg')
test_eq(dhdl.contents(), 'gggg')


In [ ]:
dhdl.update('hhhh')
test_eq(dhdl.contents(), 'hhhh')


In [ ]:
class CellRenderP(Protocol):
    def __call__(self, *o, cell_handle: CellHandle, **kwargs): ...

class CellHandle:
    def __init__(self, display_id:str, render: CellRenderP): 
        self.di = DisplayId(display_id=display_id)
        self.render = render
    def __call__(self, *args, **kwargs):
        return self.render(*args, cell_handle=self, **kwargs)


# Colophon
----


In [ ]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [ ]:
if FC.IN_NOTEBOOK:
    nb_path = '15_cell_display.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
